In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline, UnivariateSpline, InterpolatedUnivariateSpline, interp1d
from datetime import datetime
from math import floor
import matplotlib.pyplot as plt
from BEE_WEATHER_DATA import BROODMINDER_GET, AMBIENT_GET, READ_HIVE, PROCESS_HIVE, READ_BEE_WEATHER, PROCESS_BEE_WEATHER, PROCESS_AMBIENT, GRAPH_DATA, GET_MOON_IMAGE, GET_FORECAST

hive_name = "New Left Hive"
key = 0

if key == 1:
    Data = PROCESS_AMBIENT()
    #display(Data)
    GRAPH_DATA(Data)
if key == 2:
    Data = PROCESS_BEE_WEATHER()
    #display(Data)
    GRAPH_DATA(Data)
if key == 3:
    Data = PROCESS_HIVE(hive_name, interp=1)
    #display(Data)
    [GRAPH_DATA(Data[str(i)]) for i in Data.keys()]

#pix = GET_MOON_IMAGE(216, save=1)
#plt.imshow(pix)
#plt.show()

data = GET_FORECAST()
print(data)

print("Done")

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1710169200, 'main': {'temp': 45.32, 'feels_like': 40.91, 'temp_min': 44.62, 'temp_max': 45.32, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 867, 'humidity': 50, 'temp_kf': 0.39}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'clouds': {'all': 79}, 'wind': {'speed': 8.34, 'deg': 160, 'gust': 16.04}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2024-03-11 15:00:00'}, {'dt': 1710180000, 'main': {'temp': 48.7, 'feels_like': 44.49, 'temp_min': 48.7, 'temp_max': 50.23, 'pressure': 1011, 'sea_level': 1011, 'grnd_level': 867, 'humidity': 47, 'temp_kf': -0.85}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'clouds': {'all': 87}, 'wind': {'speed': 9.84, 'deg': 207, 'gust': 17.49}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2024-03-11 18:00:00'}, {'dt': 1710190800, 'main': {'temp': 50.45, 'feels_like': 47.5,